## Budko plot and US maps based on NLDAS-2 results (US)
### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pickle

import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

### Load metrics (NSE) of model 2A (NLDAS-2, HydroMT)

In [2]:
# daily
metrics_dir = 'data/results/01_nldas2_hydromt/metrics_df_daily.pkl'
with open(metrics_dir, 'rb') as fp:
    metrics_df_daily = pickle.load(fp)
metrics_df_daily = metrics_df_daily[['NSE', 'KGE', 'FHV', 'Peak-Timing']]

metrics_dir = 'data/results/01_nldas2_hydromt/metrics_df_daily_peak_error_era5.pkl'
with open(metrics_dir, 'rb') as fp:
    metrics_df_daily_peak = pickle.load(fp)

metrics_df_daily = metrics_df_daily.join(metrics_df_daily_peak)
    
# hourly
metrics_dir = 'data/results/01_nldas2_hydromt/metrics_df_hourly.pkl'
with open(metrics_dir, 'rb') as fp:
    metrics_df_hourly = pickle.load(fp)
metrics_df_hourly = metrics_df_hourly[['NSE', 'KGE', 'FHV', 'Peak-Timing']]
    
metrics_dir = 'data/results/01_nldas2_hydromt/metrics_df_hourly_peak_error_era5.pkl'
with open(metrics_dir, 'rb') as fp:
    metrics_df_hourly_peak = pickle.load(fp)

metrics_df_hourly = metrics_df_hourly.join(metrics_df_hourly_peak)

# metrics_df_hourly
# metrics_df_daily

### Loading hydrological signature, CAMELS attributes and scaling
1. Load list of 516 catchments
2. Load hydrologcial signatures (CAMELS US dataset)
3. Load attributes (CAMELS US dataset)
5. Load HRU number (CAMELS US dataset)

In [11]:
# 1. Load 516 CAMELS US catchments with hourly Qobs available
with open('data/516_basins.txt', 'r') as f:
    basin_list = [line.rstrip() for line in f]
    f.close()

# 2. Load hydrological signatures
hydrol_attrib = pd.read_csv('data/camels_attributes/camels_hydro.txt', sep=';', dtype={'gauge_id': str})
hydrol_attrib.index = hydrol_attrib.gauge_id
hydrol_attrib = hydrol_attrib.drop('gauge_id', axis=1)
hydrol_attrib = hydrol_attrib.loc[hydrol_attrib.index.isin(basin_list)]

# 3. Load attributes
attributes_df = pd.read_csv('data/camels_attributes/camels_attributes.csv', header=0, dtype={'basin': str})
attributes_df.index = attributes_df.basin
attributes_df = attributes_df.drop(columns='basin')
attributes_df = attributes_df.loc[attributes_df.index.isin(basin_list)]

# 4. Load HRU associated with each catchment
huc_df = pd.read_csv('data/camels_attributes/camels_name.txt', header=0, dtype={'gauge_id': str, 'huc_02': str}, sep=';')
huc_df.index = huc_df.gauge_id
huc_df = huc_df.loc[huc_df.index.isin(basin_list)]

# 5. Load lat/lon data for each catchment
topo_df = pd.read_csv('data/camels_attributes/camels_topo.txt', sep=';', dtype={'gauge_id': str})
topo_df.index = topo_df.gauge_id
topo_df = topo_df[topo_df.index.isin(metrics_df_daily.index)]
mfilter = topo_df['gauge_id'] == huc_df['gauge_id']
topo_df.loc[mfilter, 'huc'] = huc_df.loc[mfilter, "huc_02"]

huc_df = huc_df.drop(columns='gauge_id')

### Budyko curve
1. Load cluster information
2. Build dataframe, preprae Budyko plot
3. Plot interactive Budyko Framework

In [12]:
# 1. Load dict with catchment IDs per cluster (built on Camels US catchment attribtues)
with open('data/basins_in_cluster_camels.p', 'rb') as f:
    clusters = pickle.load(f)

# add 'Cluster' column to attribute_df
attributes_df['Cluster'] = 0
for key, item in clusters[7].items():
    attributes_df['Cluster'].loc[attributes_df.index.isin(item)] = int(key) 

C:\Users\wilbrand\Miniconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
# 2. Prepare dataframe for plotting in Budyko Framework

# budyko curve x and y values
PET_P = np.arange(0.001, 5, 0.05)   # x-axis
ET_P = (PET_P * np.tanh(1 / PET_P) * (1 - np.exp(-PET_P))) ** 0.5   # y-axis

# water limit
water_limit_x = PET_P[PET_P > 1]
water_limit_y = 1 + water_limit_x * 0

# energy limit
energy_limit_x = np.arange(0, 1.0001, 0.05)
energy_limit_y = energy_limit_x


# combine cluster info with attributes and hydrol signatures
budyko_df = pd.concat([ attributes_df, hydrol_attrib], join='inner', axis=1)

# get x and y values for each catchment
budyko_df['PET/P'] = budyko_df['pet_mean'] / (budyko_df['p_mean'])
budyko_df['ET/P'] = 1 - budyko_df['q_mean'] / (budyko_df['p_mean'])

# add 'Cluster' as str in dataframe, add column with gauge id for hover text, sort by cluster
budyko_df['Cluster'] = (budyko_df.Cluster + 1).astype(str)
budyko_df['gauge_id'] = budyko_df.index
budyko_df = pd.concat([budyko_df, huc_df], axis=1, join='inner')
budyko_df = pd.concat([budyko_df, metrics_df_daily], axis=1, join='inner')
budyko_df['NSE_scaled'] = (budyko_df.NSE - budyko_df.NSE.min()) / (budyko_df.NSE.max() - budyko_df.NSE.min()) * 100 
budyko_df = budyko_df.sort_values(by='Cluster', axis=0)

# set all negative NSE values to 0
budyko_df[budyko_df["NSE"] < 0] = 0

# budyko_df.head()

In [14]:
# 3. Budyko Plot

fig = px.scatter(budyko_df.loc[budyko_df.Cluster.isin(['1', '2', '3', '4', '5', '6', '7']) ], 
                 x = "PET/P", 
                 y = "ET/P", 
                 hover_name=budyko_df.loc[budyko_df.Cluster.isin(['1', '2', '3', '4', '5', '6', '7']) ].index,
                 custom_data=['gauge_id', 'area_gages2', 'frac_snow_daily', 'q_mean', 'elev_mean', 
                              'hfd_mean', 'huc_02', 'NSE'],
                 color = "NSE", 
                 color_continuous_scale=px.colors.sequential.Viridis,
                 symbol = budyko_df.loc[budyko_df.Cluster.isin(['1', '2', '3', '4', '5', '6', '7']) ]['Cluster']
                )

# Hovertemplate and Marker
fig.update_traces(hovertemplate='<br>'.join([
                        "catchment: %{customdata[0]} - huc: %{customdata[7]}",
                        "NSE: %{customdata[8]:.2f}",
                        "Qmean: %{customdata[4]:.2f} mm/d",
                        "Evlev. mean: %{customdata[5]:.2f m}",
                        "HFD mean: day %{customdata[6]:.2f}"]),
                 marker=dict(
                     size=10,
                     opacity=0.7,
                     line=dict(width=1,
                     color='DarkSlateGrey'))
                 )
# Budyko curve
fig.add_trace(go.Scatter(x=PET_P,
                         y=ET_P,
                         mode='lines',
                         name='Budyko curve',
                         marker_color='#0D2A63'
))

# Energy limit
fig.add_trace(go.Scatter(x=energy_limit_x,
                         y=energy_limit_y,
                         mode='lines',
                         name='Energy limit',
                         marker_color='#D62728'
))

# Water limit
fig.add_trace(go.Scatter(x=water_limit_x,
                         y=water_limit_y,
                         mode='lines',
                         name='Water limit',
                         marker_color='blue'
))

# orientation lines
fig.add_vline(x=1,
             line_dash='dash',
             line_width=1)
fig.add_hline(y=0,
             line_dash='dash',
             line_width=1)

# Layout
fig.update_xaxes(
    showspikes=True, 
    spikethickness=1, 
    linecolor='black',
    linewidth=2,
    mirror=True,
    tickfont = {"size": 20},
    title_font = {"size": 20})

fig.update_yaxes(
    showspikes=True, 
    spikethickness=1, 
    linecolor='black',
    linewidth=2,
    mirror=True,
    tickfont = {"size": 20},
    title_font = {"size": 20})

fig.update_layout(title_text=f'US catchments - NSE from model 2A (ERA5, HydroMT)',
                  paper_bgcolor='white', 
                  plot_bgcolor='white',
                  legend=dict(
                        yanchor="bottom",
                        y=0.01,
                        xanchor="right",
                        x=0.99,
                        font={'size': 15},
                        bordercolor="Black",
                        borderwidth=2,
                        title='Cluster:'
))

fig.show()
print('Click on legend entries do disable/enable Clusters shown in plot')

Click on legend entries do disable/enable Clusters shown in plot


### Plot interactive US maps with NSE per catchment
1. daily NSE
2. daily peak timing error
3. hourly NSE
4. hourly peak timing error

In [15]:
time_scale = ['daily', 'hourly']
colorscale = ['viridis_r', 'viridis']
cmin = 0

for i, metrics_df in enumerate([metrics_df_daily, metrics_df_hourly]):
    cmax = 1.0
    colorbar_title = 'NSE'
    for j, column in enumerate(['NSE', 'Peak-Timing']):

        topo_df['text'] = topo_df['huc'].astype(str) + ', basin: ' + topo_df['gauge_id'].astype(str) + ', metric:' + round(metrics_df[column], 2).astype(str)

        if j == 1 and i == 0 : 
            cmax = 3.0
            colorbar_title = '[days]'
        elif j == 1 and i == 1: 
            cmax = 12.0
            colobar_title= '[hours]'

        fig = go.Figure(data=go.Scattergeo(
                lon = topo_df['gauge_lon'],
                lat = topo_df['gauge_lat'],
                text = topo_df['text'],
                mode = 'markers',
                marker = dict(
                    size = 8,
                    opacity = 0.8,
                    reversescale = True,
                    autocolorscale = False,
                    line = dict(
                        width=1,
                        color='rgba(102, 102, 102)'
                    ),
                    colorscale = colorscale[j],
                    cmin = 0,
                    color = metrics_df[column],
                    cmax = cmax,
                    colorbar_title=colorbar_title
          )))

        fig.update_layout(
                title = f'{time_scale[i]} {column}',
                geo = dict(
                    scope='usa',
                    projection_type='albers usa',
                    showland = True,
                    landcolor = "rgb(250, 250, 250)",
                    subunitcolor = "rgb(217, 217, 217)",
                    countrycolor = "rgb(217, 217, 217)",
                    countrywidth = 0.5,
                    subunitwidth = 0.5
                ),
            )
        fig.show()